# Preprocessing Flux Distributions for Objective III #

In [1]:
import numpy as np
import cameo
import pandas as pd
from copy import deepcopy
import pickle
import time
import multiprocessing
from joblib import Parallel, delayed
from mcpecaso.plotting import multiplot_envelopes,plot_envelope,plot_pecaso_dfba,\
                              multi_two_stage_char_contours,two_stage_char_contour
from mcpecaso.core import mcPECASO
import mcpecaso as mcpecaso
from mcpecaso.core.substrate_dependent_envelopes import linear_uptake, logistic_uptake
import time


mcpecaso.settings.time_end = 100
mcpecaso.settings.initial_biomass = 0.05
mcpecaso.settings.initial_substrate = 500
mcpecaso.settings.num_timepoints = 10000
mcpecaso.settings.scope = 'extrema'

pd.options.display.max_rows = 100

num_cores = multiprocessing.cpu_count()

In [2]:
import sys
import colorlover as cl
from plotly import tools, subplots
import plotly.graph_objs as go
import pickle
import plotly.io as pio
pio.templates.default = "none"
import os

try:
    _ = __IPYTHON__
except NameError:
    from plotly.offline import plot
else:
    if 'ipykernel' in sys.modules:
        from plotly.offline import init_notebook_mode
        from plotly.offline import iplot as plot
        from IPython.display import HTML
        HTML("""
             <script>
              var waitForPlotly = setInterval( function() {
              if( typeof(window.Plotly) !== "undefined" ){
              MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });
              MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);
              clearInterval(waitForPlotly);}}, 250 );
            </script>
            """
        )
        init_notebook_mode(connected=True)
    elif 'IPython' in sys.modules:
        from plotly.offline import plot
    else:
        warn('Unknown ipython configuration')
        from plotly.offline import plot


In [3]:
pd.options.display.max_rows=600
pd.options.display.max_columns=150
wt_model = cameo.models.bigg.iJO1366

relevant_reactions = [reaction.id for reaction in wt_model.reactions 
                      if reaction.subsystem not in ['Transport, Outer Membrane',
                                                    'Transport, Inner Membrane',
                                                    'Inorganic Ion Transport and Metabolism',
                                                    'Transport, Outer Membrane Porin',
                                                    'Intracellular demand',
                                                    'Extracellular exchange']]
relevant_reactions_set = set(relevant_reactions)

metabolite_names_dict = {'Spermidine': 'Spermidine',
                         'L-Tryptophan': 'L-Tryptophan',
                         'Thymidine C10H14N2O5': 'Thymidine',
                         'L-Phenylalanine': 'L-Phenylalanine',
                         'Adenosine': 'Adenosine',
                         'Indole': 'Indole',
                         'L-Tyrosine': 'L-Tyrosine',
                         'Inosine': 'Inosine',
                         'Xanthosine': 'Xanthosine',
                         'Cytidine': 'Cytidine',
                         'Uridine': 'Uridine',
                         'Quinate': 'Quinate',
                         'L-Isoleucine': 'L-Isoleucine',
                         'L-Lysine': 'L-Lysine',
                         'Hexanoate (n-C6:0)': 'Hexanoate',
                         'L-Leucine': 'L-Leucine',
                         'L-Arginine': 'L-Arginine',
                         'L-Histidine': 'L-Histidine',
                         'D-Gluconate': 'D-Gluconic Acid',
                         'L-Idonate': 'L-Idonic Acid',
                         '5-Dehydro-D-gluconate': '5-Ketogluconate',
                         'Citrate': 'Citric Acid',
                         'Agmatine': 'Agmatine',
                         'Ornithine': 'Ornithine',
                         'L-Proline': 'L-Proline',
                         'L-Valine': 'L-Valine',
                         'Thymine C5H6N2O2': 'Thymine',
                         'Adenine': 'Adenine',
                         'Guanine': 'Guanine',
                         'Hypoxanthine': 'Hypoxanthine',
                         'O-Acetyl-L-serine': 'O-Acetyl-L-serine',
                         'Xanthine': 'Xanthine',
                         'L-Glutamate': 'L-Glutamate',
                         '2-Oxoglutarate': '\u03b1-Ketoglutarate',
                         'Putrescine': 'Putrescine',
                         'L-Threonine': 'L-Threonine',
                         '4-Aminobutanoate': '\u03b3-Aminobutyrate',
                         'L-Homoserine': 'L-Homoserine',
                         'Allantoin': 'Allantoin',
                         'Succinate': 'Succinate',
                         'Uracil': 'Uracil',
                         'L-Asparagine': 'L-Asparagine',
                         'L-Malate': 'L-Malate',
                         'L-Aspartate': 'L-Aspartate',
                         'L-Cysteine': 'L-Cysteine',
                         'Glycerol 3-phosphate': 'Glycerol3-phosphate',
                         '3-Hydroxypropanoate': '3-Hydroxypropanoate',
                         '(S)-Propane-1,2-diol': '(S)-Propanediol',
                         '(R)-Propane-1,2-diol': '(R)-Propanediol',
                         'D-Alanine': 'D-Alanine',
                         'Glycerol': 'Glycerol',
                         '(R)-Glycerate': '(R)-Glycerate',
                         'D-Glyceraldehyde': 'Glyceraldehyde',
                         'L-Lactate': 'L-Lactate',
                         'Dihydroxyacetone': 'Dihydroxyacetone',
                         'L-Alanine': 'L-Alanine',
                         'D-Lactate': 'D-Lactate',
                         'L-Serine': 'L-Serine',
                         'Pyruvate': 'Pyruvate',
                         'Ethanolamine': 'Ethanolamine',
                         'Ethanol': 'Ethanol',
                         'Acetaldehyde': 'Acetaldehyde',
                         'Glycine': 'Glycine',
                         'Acetate': 'Acetate',
                         'Glycolate C2H3O3': 'Glycolate',
                         'Urea CH4N2O': 'Urea',
                         'Formate': 'Formate',
                         'Reduced glutathione': 'Glutathione',
                         '5-Methylthio-D-ribose': '5-Methylthioribose',
                         '1,5-Diaminopentane': 'Cadaverine'}

model = cameo.models.bigg.iJO1366
model.solver = 'gurobi'

metabolite_lookup_dict = {metabolite_names_dict[key]:key for key in metabolite_names_dict}

carbon_dict = {metabolite.name:str(metabolite.elements['C'])
               if 'C' in metabolite.elements else '0'
               for metabolite in model.metabolites}

carbon_dict = {metabolite:carbon_dict[metabolite] 
               if int(carbon_dict[metabolite])<=6 else '>6'
               for metabolite in metabolite_names_dict}


aa_metabolism_subsystems=['Alanine and Aspartate Metabolism',
                          'Arginine and Proline Metabolism',
                          'Cysteine Metabolism',
                          'Glutamate Metabolism',
                          'Glycine and Serine Metabolism',
                          'Histidine Metabolism',
                          'Methionine Metabolism',
                          'Threonine and Lysine Metabolism',
                          'Tyrosine, Tryptophan, and Phenylalanine Metabolism',
                          'Valine, Leucine, and Isoleucine Metabolism']
ijo_sub = {reaction.id:reaction.subsystem if reaction.subsystem not in aa_metabolism_subsystems
                                                    else 'Amino Acid Metabolism'
           for reaction in model.reactions}


Academic license - for non-commercial use only


In [4]:
pickling_off = open("flux_dicts.pickle","rb")
flux_dicts = pickle.load(pickling_off)
pickling_off.close()

In [5]:
flux_dict_prod = flux_dicts['productivity']
flux_dict_prod_75_yield = flux_dicts['productivity_75_yield']
flux_dict_yield_2_prod = flux_dicts['yield_2_productivity']

In [10]:
def reduced_fva(flux_dict, product, used_reactions, unused_reactions, template_model, slack=0):
    
    model = deepcopy(template_model)
    if flux_dict[product]:
        biomass_flux, substrate_flux, target_flux = flux_dict[product]
        target_rxn = [reaction for reaction in model.exchanges 
                      if reaction.reactants[0].name==metabolite_lookup_dict[product]][0]

        with model:
            for reaction in list(unused_reactions):
                    if model.reactions.get_by_id(reaction).reversibility:
                        model.reactions.get_by_id(reaction).bounds = (-slack, slack)
                    else:
                        model.reactions.get_by_id(reaction).bounds = (0, slack)

             #CONSTRAINTS
            model.reactions.EX_glc__D_e.bounds = (substrate_flux-slack, 1000)
            model.reactions.get_by_id(target_rxn.id).bounds = (target_flux-slack, target_flux+slack)

            mut_df = cameo.flux_variability_analysis(model,fraction_of_optimum=1,reactions=used_reactions).data_frame
            mut_pfba = cameo.pfba(model).data_frame.reindex(used_reactions)

        mut_pfba[mut_pfba.abs()<0.001]=0
        mut_df[mut_df.abs()<0.001]=0
        temp = pd.concat([mut_pfba,mut_df], axis=1, sort=True)
        temp.columns = ['pfba','lb','ub']
        temp['range'] = temp.ub-temp.lb
        temp[temp.abs()<0.001]=0

        return {product:temp}
    else:
        return{product:None}

In [11]:
def global_fva_parser(fva_dict, flex_cutoff=0.001):
    parsed_fva = {}
    temp_dict = deepcopy(fva_dict)
    for product in temp_dict.keys():
        temp_dict[product] = temp_dict[product].fillna(0)
        pfba_values = temp_dict[product][(temp_dict[product].range<=flex_cutoff)].pfba.values
        temp_dict[product].loc[temp_dict[product][(temp_dict[product].range<=flex_cutoff)].index,'lb'] = pfba_values
        temp_dict[product].loc[temp_dict[product][(temp_dict[product].range<=flex_cutoff)].index,'ub'] = pfba_values
        temp_dict[product].loc[temp_dict[product][(temp_dict[product].range<=flex_cutoff)].index,'range'] = 0
    
    
    products = list(temp_dict.keys())

    range_df = pd.concat([temp_dict[product].range for product in temp_dict.keys()],axis=1)
    range_df = range_df.fillna(0)
    flexible_reactions_set = set(range_df[range_df.any(axis=1)].index)
    inflexible_reactions_set = relevant_reactions_set - flexible_reactions_set
    
    
    pfba_df = pd.concat([temp_dict[product].pfba for product in temp_dict.keys()],axis=1)
    pfba_df = pfba_df.fillna(0)
    pfba_df.columns = products
    
    used_reactions_set = set(pfba_df[pfba_df.any(axis=1)].index)
    unused_reactions_set = relevant_reactions_set - used_reactions_set

    parsed_fva['used_reactions_set'] = used_reactions_set
    parsed_fva['unused_reactions_set'] = unused_reactions_set
    parsed_fva['flexible_reactions_set'] = flexible_reactions_set
    parsed_fva['inflexible_reactions_set'] = inflexible_reactions_set
    parsed_fva['inflexible'] = {}
    parsed_fva['inflexible']['pfba'] = pd.concat([temp_dict[product].reindex(inflexible_reactions_set.intersection(used_reactions_set)).pfba for product in temp_dict.keys()],axis=1)
    parsed_fva['inflexible']['pfba'].columns = products
    parsed_fva['flexible'] = {}
    parsed_fva['flexible']['pfba'] = pd.concat([temp_dict[product].reindex(flexible_reactions_set.intersection(used_reactions_set)).pfba for product in temp_dict.keys()],axis=1)
    parsed_fva['flexible']['pfba'].columns = products
    columns = []
    for product in temp_dict.keys():
        columns.append((product,'lb'))
        columns.append((product,'ub'))
    columns = pd.MultiIndex.from_tuples(columns)
    parsed_fva['flexible']['bounds'] = pd.concat([temp_dict[product].reindex(flexible_reactions_set.intersection(used_reactions_set))[['lb','ub']] for product in temp_dict.keys()], axis=1)
    parsed_fva['flexible']['bounds'].columns = columns
    parsed_fva['flexible']['range'] = pd.concat([temp_dict[product].reindex(flexible_reactions_set.intersection(used_reactions_set)).range for product in temp_dict.keys()],axis=1)
    parsed_fva['flexible']['range'].columns = products
    
    #products flexible, products inflexible but active, products inflexible AND inactive
    flex_products = []
    flex_products_no = []
    inflex_used_products = []
    inflex_used_products_no = []
    inflex_unused_products = []
    inflex_unused_products_no = []
    for reaction in parsed_fva['flexible']['pfba'].index:
        flex_products.append(set(parsed_fva['flexible']['range'].loc[reaction][parsed_fva['flexible']['range'].loc[reaction]!=0].index))
        flex_products_no.append(len(parsed_fva['flexible']['range'].loc[reaction][parsed_fva['flexible']['range'].loc[reaction]!=0].index))
        inflex_products = parsed_fva['flexible']['range'].loc[reaction][parsed_fva['flexible']['range'].loc[reaction]==0].index
        inflex_used_products.append(set(parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)[parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)!=0].index))
        inflex_used_products_no.append(len(parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)[parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)!=0].index))
        inflex_unused_products.append(set(parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)[parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)==0].index))
        inflex_unused_products_no.append(len(parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)[parsed_fva['flexible']['pfba'].loc[reaction].reindex(inflex_products)==0].index))

    parsed_fva['flexible']['products'] = pd.DataFrame(data={'flex':flex_products, 'flex_no':flex_products_no, 'inflex_used':inflex_used_products,
                                                 'inflex_used_no':inflex_used_products_no, 'inflex_unused':inflex_unused_products,
                                                 'inflex_unused_no':inflex_unused_products_no}, index=parsed_fva['flexible']['pfba'].index)

    
    sorted_order = parsed_fva['flexible']['products'].sort_values(by=['flex_no','inflex_used_no'],ascending=[False,False]).index
    parsed_fva['flexible']['pfba'] = parsed_fva['flexible']['pfba'].reindex(sorted_order)
    parsed_fva['flexible']['bounds'] = parsed_fva['flexible']['bounds'].reindex(sorted_order) 
    parsed_fva['flexible']['range'] = parsed_fva['flexible']['range'].reindex(sorted_order)
    parsed_fva['flexible']['products'] = parsed_fva['flexible']['products'].reindex(sorted_order) 
    
    
    return temp_dict,parsed_fva

# FIRST PASS FVA #

In [30]:
#FIRST PASS FVA - NO EXTRA CONSTRAINTS APART FROM WILDTYPE. FVA DONE ONLY ON RELEVANT REACTIO
model.solver='gurobi'

start_time = time.time()
result_list = Parallel(n_jobs=num_cores, verbose=20)(delayed(reduced_fva)(flux_dict_yield_2_prod, product, relevant_reactions_set,
                                                                         [], model, slack=0) for product in flux_dict_yield_2_prod.keys())


fva = {list(result_dict.keys())[0]: list(result_dict.values())[0] 
                                for result_dict in result_list}

print(time.time()-start_time)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:   43.2s
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:  1.3min
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:  1.9min
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:  2.0min
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:  2.0min
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:  2.2min
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:  2.3min
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:  2.3min
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:  2.5min
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:  2.9min
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:  3.5min
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:  3.5min
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:  3.8min
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:  4.0min
[Parallel(n_jobs=8)]: Done  15 tasks      | elapsed:  4.0min
[Parallel(

885.8002555370331


[Parallel(n_jobs=8)]: Done  70 out of  70 | elapsed: 14.8min finished


In [38]:
fva = {key:fva[key] for key in fva.keys() if fva[key] is not None}


In [39]:
pickling_on = open("fva_yield_2_prod.pickle","wb")
pickle.dump(fva, pickling_on)
pickling_on.close() 

In [40]:
pickle_off = open("fva_yield_2_prod.pickle","rb")
fva = pickle.load(pickle_off)
pickle_off.close()

In [41]:
reactions_set = set([reaction.id for reaction in model.reactions])
relevant_reactions_set = set(relevant_reactions)
irrelevant_reactions_set = reactions_set-relevant_reactions_set

fva,parsed_fva = global_fva_parser(fva,flex_cutoff=0.001)

print("Total Reactions: ",len(reactions_set), "\n")
print("Irrelevant_reactions: ",len(irrelevant_reactions_set))
print("Relevant Reactions: ",len(relevant_reactions_set), "\n")
print("Inflexible Reactions: ",len(parsed_fva['inflexible_reactions_set']))
print("Unused Inflexible Reactions: ",len(parsed_fva['inflexible_reactions_set'].intersection(parsed_fva['unused_reactions_set'])))
print("Used Inflexible Reactions: ",len(parsed_fva['inflexible_reactions_set'].intersection(parsed_fva['used_reactions_set'])), "\n")
print("Flexible Reactions: ",len(parsed_fva['flexible_reactions_set']))
print("Unused Flexible Reactions: ",len(parsed_fva['flexible_reactions_set'].intersection(parsed_fva['unused_reactions_set'])))
print("Used Flexible Reactions: ",len(parsed_fva['flexible_reactions_set'].intersection(parsed_fva['used_reactions_set'])))

unused_reactions_set_1 = parsed_fva['unused_reactions_set']
used_reactions_set_1 = parsed_fva['used_reactions_set']

Total Reactions:  2583 

Irrelevant_reactions:  1090
Relevant Reactions:  1493 

Inflexible Reactions:  1192
Unused Inflexible Reactions:  973
Used Inflexible Reactions:  219 

Flexible Reactions:  301
Unused Flexible Reactions:  136
Used Flexible Reactions:  165


In [42]:
wt_model.solver = 'gurobi'
wt_df = cameo.flux_variability_analysis(wt_model,fraction_of_optimum=1,reactions=relevant_reactions).data_frame
wt_pfba = cameo.pfba(wt_model).data_frame.reindex(relevant_reactions)

#Pruning out small values. If a flux is less than 0.001, make it equal to 0
wt_pfba[wt_pfba.abs()<0.001]=0
wt_df[wt_df.abs()<0.001]=0

#Consolidate pfba and fva into a single dataframe
comparison_df = pd.concat([wt_pfba,wt_df], axis=1, sort=True)
comparison_df.columns = ['pfba','lb','ub']

#Add a column describing the range of allowable fluxes for each reaction
comparison_df['range'] = comparison_df.ub-comparison_df.lb
wt_fva = comparison_df
wt_used_reactions = set(list(wt_fva[wt_fva.pfba!=0].index))

# SECOND PASS FVA #

In [43]:
#SECOND PASS FVA - REACTIONS UNUSED FOR ALL PRODUCTS FROM FIRST PASS CONSTRAINED TO ZERO
start_time = time.time()

result_list = Parallel(n_jobs=num_cores, verbose=20)(delayed(reduced_fva)(flux_dict_yield_2_prod, product, used_reactions=used_reactions_set_1,
                                                                         unused_reactions=unused_reactions_set_1,
                                                                          template_model=model, slack=0.001) for product in flux_dict_yield_2_prod.keys())


fva_second = {list(result_dict.keys())[0]: list(result_dict.values())[0] 
                                for result_dict in result_list}

print("Completed in ", time.time()-start_time, " s")

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:   36.2s
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   54.3s
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:   56.8s
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:   56.8s
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:   59.1s
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:  1.1min
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:  1.3min
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:  1.3min
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:  1.5min
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:  1.7min
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:  1.7min
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:  1.9min
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:  1.9min
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:  1.9min
[Parallel(n_jobs=8)]: Done  15 tasks      | elapsed:  2.1min
[Parallel(

Completed in  460.9529104232788  s


[Parallel(n_jobs=8)]: Done  70 out of  70 | elapsed:  7.7min finished


In [44]:
pickling_on = open("fva_second_yield_2_prod.pickle","wb")
pickle.dump(fva_second, pickling_on)
pickling_on.close() 

In [45]:
pickle_off = open("fva_second_yield_2_prod.pickle","rb")
fva_second = pickle.load(pickle_off)
pickle_off.close()

In [46]:
fva_second = {key:fva_second[key] for key in fva_second.keys() if fva_second[key] is not None}

In [47]:
reactions_set = set([reaction.id for reaction in model.reactions])
relevant_reactions_set = set(relevant_reactions)
irrelevant_reactions_set = reactions_set-relevant_reactions_set

fva_second,parsed_fva_2 = global_fva_parser(fva_second,flex_cutoff=0.008)

print("Total Reactions: ",len(reactions_set), "\n")
print("Irrelevant_reactions: ",len(irrelevant_reactions_set))
print("Relevant Reactions: ",len(relevant_reactions_set), "\n")
print("Inflexible Reactions: ",len(parsed_fva_2['inflexible_reactions_set']))
print("Unused Inflexible Reactions: ",len(parsed_fva_2['inflexible_reactions_set'].intersection(parsed_fva_2['unused_reactions_set'])))
print("Used Inflexible Reactions: ",len(parsed_fva_2['inflexible_reactions_set'].intersection(parsed_fva_2['used_reactions_set'])), "\n")
print("Flexible Reactions: ",len(parsed_fva_2['flexible_reactions_set']))
print("Unused Flexible Reactions: ",len(parsed_fva_2['flexible_reactions_set'].intersection(parsed_fva_2['unused_reactions_set'])))
print("Used Flexible Reactions: ",len(parsed_fva_2['flexible_reactions_set'].intersection(parsed_fva_2['used_reactions_set'])))

unused_reactions_set_2 = parsed_fva_2['unused_reactions_set']
used_reactions_set_2 = parsed_fva_2['used_reactions_set']

Total Reactions:  2583 

Irrelevant_reactions:  1090
Relevant Reactions:  1493 

Inflexible Reactions:  1406
Unused Inflexible Reactions:  1113
Used Inflexible Reactions:  293 

Flexible Reactions:  87
Unused Flexible Reactions:  1
Used Flexible Reactions:  86


In [29]:
parsed_fva_2['flexible']['products']

,flex,flex_no,inflex_used,inflex_used_no,inflex_unused,inflex_unused_no
PRPPS,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
R1PK,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
FBA,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
TALA,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
R15BPK,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
PPM,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
NADH16pp,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
DHORD2,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
ADK1,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
DHORDfum,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0


# THIRD PASS FVA #

In [49]:
#THIRD PASS FVA - REACTIONS UNUSED FOR ALL PRODUCTS FROM SECOND PASS CONSTRAINED TO ZERO
start_time = time.time()

result_list = Parallel(n_jobs=num_cores, verbose=20)(delayed(reduced_fva)(flux_dict_yield_2_prod, product, used_reactions=used_reactions_set_2,
                                                                         unused_reactions=unused_reactions_set_2,
                                                                          template_model=model, slack=0.001) for product in flux_dict_yield_2_prod.keys())


fva_third = {list(result_dict.keys())[0]: list(result_dict.values())[0] 
                                for result_dict in result_list}

print("Completed in ", time.time()-start_time, " s")

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:   22.4s
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   42.9s
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:   48.2s
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:   48.3s
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:   49.6s
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:  1.0min
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:  1.1min
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:  1.2min
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:  1.3min
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:  1.6min
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:  1.7min
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:  1.8min
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:  1.8min
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:  2.0min
[Parallel(n_jobs=8)]: Done  15 tasks      | elapsed:  2.2min
[Parallel(

Completed in  493.3934533596039  s


[Parallel(n_jobs=8)]: Done  70 out of  70 | elapsed:  8.2min finished


In [52]:
pickling_on = open("fva_third_yield_2_prod.pickle","wb")
pickle.dump(fva_third, pickling_on)
pickling_on.close()

In [55]:
pickle_off = open("fva_third_yield_2_prod.pickle","rb")
fva_third = pickle.load(pickle_off)
pickle_off.close()

In [56]:
fva_third = {key:fva_third[key] for key in fva_third.keys() if fva_third[key] is not None}

In [57]:
reactions_set = set([reaction.id for reaction in model.reactions])
relevant_reactions_set = set(relevant_reactions)
irrelevant_reactions_set = reactions_set-relevant_reactions_set

fva_third_temp,parsed_fva_3 = global_fva_parser(fva_third,flex_cutoff=0.008)

print("Total Reactions: ",len(reactions_set), "\n")
print("Irrelevant_reactions: ",len(irrelevant_reactions_set))
print("Relevant Reactions: ",len(relevant_reactions_set), "\n")
print("Inflexible Reactions: ",len(parsed_fva_3['inflexible_reactions_set']))
print("Unused Inflexible Reactions: ",len(parsed_fva_3['inflexible_reactions_set'].intersection(parsed_fva_3['unused_reactions_set'])))
print("Used Inflexible Reactions: ",len(parsed_fva_3['inflexible_reactions_set'].intersection(parsed_fva_3['used_reactions_set'])), "\n")
print("Flexible Reactions: ",len(parsed_fva_3['flexible_reactions_set']))
print("Unused Flexible Reactions: ",len(parsed_fva_3['flexible_reactions_set'].intersection(parsed_fva_3['unused_reactions_set'])))
print("Used Flexible Reactions: ",len(parsed_fva_3['flexible_reactions_set'].intersection(parsed_fva_3['used_reactions_set'])))

unused_reactions_set_3 = parsed_fva_3['unused_reactions_set']
used_reactions_set_3 = parsed_fva_3['used_reactions_set']

Total Reactions:  2583 

Irrelevant_reactions:  1090
Relevant Reactions:  1493 

Inflexible Reactions:  1412
Unused Inflexible Reactions:  1114
Used Inflexible Reactions:  298 

Flexible Reactions:  81
Unused Flexible Reactions:  0
Used Flexible Reactions:  81


In [60]:
parsed_fva_3['flexible']['products']

,flex,flex_no,inflex_used,inflex_used_no,inflex_unused,inflex_unused_no
PRPPS,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
R1PK,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
FBA,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
TALA,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
R15BPK,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
PPM,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
NADH16pp,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
DHORD2,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
ADK1,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
DHORDfum,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0


# FOURTH PASS FVA #

In [74]:
parsed_fva_3_temp = deepcopy(parsed_fva_3)
fva_third_temp = deepcopy(fva_third)

In [76]:
#FOURTH PASS FVA - FLEXIBLE REACTIONS THAT CAN BE ZERO FOR ALL PRODUCTS REMOVED ONE BY ONE AND FVA UPDATED AFTER EACH REMOVAL

deletable_reactions = parsed_fva_3['flexible']['products'][parsed_fva_3_temp['flexible']['products'].inflex_used_no==0].index


parsed_fva_3_temp['flexible']['products'] = parsed_fva_3_temp['flexible']['products'].sort_index().sort_values(by=['flex_no','inflex_used_no'], ascending=[False,False])

old_list = ['DHORD2', 'SUCDi', 'NADH17pp', 'R1PK', 'FRD2', 'FBA', 'R15BPK', 'TALA',
           'ADK1', 'NADH16pp', 'PRPPS', 'PPM', 'DHORDfum', 'FBA3', 'PFK', 'PFK_3',
           'HEX1', 'XYLI2', 'HEX7', 'GART', 'FTHFLi', 'THRD', 'THRA', 'RNDR3',
           'RNDR2', 'NDPK6', 'NDPK5', 'FLDR2', 'RNDR4', 'NDPK7', 'FADRx2', 'AMPN',
           'NTD7', 'TRPS1', 'IDOND', 'IDOND2', 'SERD_L', 'FBP']
new_list = [reaction for reaction in old_list if reaction in deletable_reactions]
new_list.extend(set(deletable_reactions)-set(new_list))

deletable_reactions = parsed_fva_3_temp['flexible']['products'].reindex(new_list).sort_values(by=['flex_no','inflex_used_no'],ascending=[False,False]).index


for reaction in reversed(deletable_reactions):
    if (reaction in parsed_fva_3_temp['flexible']['products'].index) and (reaction not in wt_used_reactions):
        zeroflag=1
        flex_products = [metabolite_lookup_dict[product] for product in parsed_fva_3_temp['flexible']['products']['flex'].loc[reaction]]
        for product in flex_products:
            if not((parsed_fva_3_temp['flexible']['bounds'].loc[reaction].loc[metabolite_names_dict[product]].lb<=0) and (parsed_fva_3_temp['flexible']['bounds'].loc[reaction].loc[metabolite_names_dict[product]].ub>=0)):
                zeroflag=0
        if zeroflag:
            print("Removing reaction: ", reaction)
            used_reactions_set = parsed_fva_3_temp['used_reactions_set'] - set([reaction])
            unused_reactions_set = parsed_fva_3_temp['unused_reactions_set'].union(set([reaction]))
            for i,product in enumerate(flex_products):
                print('Evaluating Product ', i+1, ' of ', len(flex_products))
                temp = reduced_fva(flux_dict_yield_2_prod, metabolite_names_dict[product],
                                                                             unused_reactions=unused_reactions_set,
                                                                             used_reactions=used_reactions_set,
                                                                             template_model=model, slack=0.001)
                fva_third_temp[metabolite_names_dict[product]] = list(temp.values())[0]
            
            fva_third_temp,parsed_fva_3_temp = global_fva_parser(fva_third_temp,flex_cutoff=0.008)


C:\Users\kraj5\Anaconda3\envs\dyssco_plotly_4\lib\site-packages\ipykernel_launcher.py:3: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



Removing reaction:  NADTRHD
Evaluating Product  1  of  2
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpuslpg5hp.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  2  of  2
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp1cfu_m6p.lp
Reading time = 0.04 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


C:\Users\kraj5\Anaconda3\envs\dyssco_plotly_4\lib\site-packages\ipykernel_launcher.py:14: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.


C:\Users\kraj5\Anaconda3\envs\dyssco_plotly_4\lib\site-packages\ipykernel_launcher.py:20: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




Removing reaction:  IDOND2
Evaluating Product  1  of  2
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmppe220bzr.lp
Reading time = 0.06 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  2  of  2
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp9_gfghwa.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  NTD7
Evaluating Product  1  of  2
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpt80i8aik.lp
Reading time = 0.04 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  2  of  2
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpulec14a2.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  FADRx2
Evaluating Product  1  of  2
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpmxxup1gu.lp
Reading time = 0.06 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  2  of 

Reading time = 0.06 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  9  of  53
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpqsdznmd4.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  10  of  53
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpi6gsp5f5.lp
Reading time = 0.04 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  11  of  53
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpm20ixc1d.lp
Reading time = 0.04 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  12  of  53
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp8ht0qwg8.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  13  of  53
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpc9n7f15d.lp
Reading time = 0.05 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  14  of

Removing reaction:  DHORDfum
Evaluating Product  1  of  67
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpq3e4uws2.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  2  of  67
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp3f8n_hwc.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  3  of  67
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpzqfvzy93.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  4  of  67
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpvtz7bng7.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  5  of  67
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmppczm4fbf.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  6  of  67
Read LP format model from file C:\Users\k

Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpvl22d1hj.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  47  of  67
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpnue4cp7c.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  48  of  67
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmprryejvc9.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  49  of  67
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp_jhzotkt.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  50  of  67
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpsbbtkahn.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  51  of  67
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpu181_yiu.lp
Reading time = 

Reading time = 0.05 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  25  of  67
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmppj2zqdkm.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  26  of  67
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmphhwpuiig.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  27  of  67
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpro3jxq76.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  28  of  67
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpwvp41jtt.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  29  of  67
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpgbteinwy.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating Product  30  o

In [77]:
fva_fourth = deepcopy(fva_third_temp)


In [78]:
pickling_on = open("fva_fourth_yield_2_prod.pickle","wb")
pickle.dump(fva_fourth, pickling_on)
pickling_on.close() 

In [89]:
pickle_off = open("fva_fourth_yield_2_prod.pickle","rb")
fva_fourth = pickle.load(pickle_off)
pickle_off.close()

In [90]:
fva_fourth = {key:fva_fourth[key] for key in fva_fourth.keys() if fva_fourth[key] is not None}

In [91]:
reactions_set = set([reaction.id for reaction in model.reactions])
relevant_reactions_set = set(relevant_reactions)
irrelevant_reactions_set = reactions_set-relevant_reactions_set

fva_fourth,parsed_fva_4 = global_fva_parser(fva_fourth,flex_cutoff=0.008)

print("Total Reactions: ",len(reactions_set), "\n")
print("Irrelevant_reactions: ",len(irrelevant_reactions_set))
print("Relevant Reactions: ",len(relevant_reactions_set), "\n")
print("Inflexible Reactions: ",len(parsed_fva_4['inflexible_reactions_set']))
print("Unused Inflexible Reactions: ",len(parsed_fva_4['inflexible_reactions_set'].intersection(parsed_fva_4['unused_reactions_set'])))
print("Used Inflexible Reactions: ",len(parsed_fva_4['inflexible_reactions_set'].intersection(parsed_fva_4['used_reactions_set'])), "\n")
print("Flexible Reactions: ",len(parsed_fva_4['flexible_reactions_set']))
print("Unused Flexible Reactions: ",len(parsed_fva_4['flexible_reactions_set'].intersection(parsed_fva_4['unused_reactions_set'])))
print("Used Flexible Reactions: ",len(parsed_fva_4['flexible_reactions_set'].intersection(parsed_fva_4['used_reactions_set'])))

unused_reactions_set_4 = parsed_fva_4['unused_reactions_set']
used_reactions_set_4 = parsed_fva_4['used_reactions_set']

Total Reactions:  2583 

Irrelevant_reactions:  1090
Relevant Reactions:  1493 

Inflexible Reactions:  1435
Unused Inflexible Reactions:  1123
Used Inflexible Reactions:  312 

Flexible Reactions:  58
Unused Flexible Reactions:  0
Used Flexible Reactions:  58


In [92]:
parsed_fva_4['flexible']['products']

,flex,flex_no,inflex_used,inflex_used_no,inflex_unused,inflex_unused_no
FRD3,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
NADH16pp,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
FBA,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
TALA,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
SUCDi,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
NADH18pp,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",67,{},0,{},0
PFK_3,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",66,{},0,{Dihydroxyacetone},1
PFK,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",66,{},0,{Dihydroxyacetone},1
FBA3,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",66,{},0,{Dihydroxyacetone},1
MDH,"{L-Idonic Acid, (R)-Propanediol, 5-Ketoglucona...",4,"{L-Leucine, L-Malate, L-Threonine, L-Tryptopha...",63,{},0


# Fifth Pass FVA - Product Specific Deletions#

In [93]:
# Identifies product specific constraints that are possible:
# From fourth pass FVA, perform following steps:
# 1. Identify which products still have flexible reactions and iterate through each:
# 2. For each flexible reaction in that product:
# 3. Count products whose PFBA result falls within the flux range for that reaction
# 4. Also, identify the most frequently occurring flux value and the number of times it appears.
# 5. Arrange in descending order of number of times the most frequently occurring flux value occurs.

complete_products_set = set(fva_fourth.keys())
flex_products_lists = list(parsed_fva_4['flexible']['products']['flex'].values)
flex_products_sets = set([frozenset(product_list) for product_list in flex_products_lists])
temp_list = []
for temp_set in flex_products_sets:
    temp_list.extend(list(temp_set))
flex_products = (set(temp_list))
temp_dict = {}
for i,product in enumerate(flex_products):
    print('Analyzing Product :', product)
    reactions = parsed_fva_4['flexible']['range'][product][parsed_fva_4['flexible']['range'][product]!=0].index
    products_within_bounds = []
    products_within_bounds_no = []
    mode_pfba = []
    mode_no =[]
    for reaction in reactions:
        ub = parsed_fva_4['flexible']['bounds'].loc[reaction][product]['ub']
        lb = parsed_fva_4['flexible']['bounds'].loc[reaction][product]['lb']
        pfba = parsed_fva_4['flexible']['pfba'].loc[reaction][complete_products_set-set([product])]
        temp_df = pfba[(pfba>=lb)&(pfba<=ub)]
        temp_df = temp_df.round(decimals=3)
        products_within_bounds.append(list(temp_df.index))
        products_within_bounds_no.append(len(temp_df.index))
        if len(temp_df.index):
            mode_pfba.append(temp_df.mode().values[0])
            mode_no.append(len(temp_df[temp_df==temp_df.mode().values[0]].index))
        else:
            mode_pfba.append(np.nan)
            mode_no.append(0)
    data = {'products_within_bounds':products_within_bounds,'number':products_within_bounds_no,'mode_pfba':mode_pfba,'mode_no':mode_no}
    df = pd.DataFrame(data,index=reactions)
    temp_df = pd.concat([parsed_fva_4['flexible']['bounds'].loc[reactions][product],parsed_fva_4['flexible']['pfba'].loc[reactions][product]],axis=1)
    product_df = pd.concat([temp_df,df],axis=1)
    product_df = product_df.sort_values(by='mode_no',ascending=False)
    product_df.columns = ['lower','upper','pfba','products_within_bounds','number_within_bounds','mode','no_mode']
    temp_dict[product]=product_df
    display(product_df)

Analyzing Product : L-Leucine


,lower,upper,pfba,products_within_bounds,number_within_bounds,mode,no_mode
FRD3,0.000000,7.627410,0.000000,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",66,0.000,65
NADH18pp,0.000000,7.627269,0.000000,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",66,0.000,65
PFK_3,0.000000,1.474578,0.000000,"[(R)-Propanediol, Glycolate, α-Ketoglutarate, ...",57,0.000,22
FBA3,0.000000,1.474578,0.000000,"[(R)-Propanediol, Glycolate, α-Ketoglutarate, ...",57,0.000,22
SUCDi,0.000000,7.629551,0.000000,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",66,0.000,16
PFK,0.000000,1.474578,1.473578,"[Glycolate, Thymine, L-Isoleucine, L-Valine, L...",22,0.000,3
FBA,0.000000,1.474578,1.473578,"[Glycolate, Thymine, L-Isoleucine, L-Valine, L...",21,0.000,2
NADH16pp,0.000000,7.627269,7.626269,"[(R)-Propanediol, Glycolate, α-Ketoglutarate, ...",29,2.765,1
TALA,-1.516187,-0.041610,-0.041610,"[Glycolate, α-Ketoglutarate, Cytidine, Acetate...",11,-0.962,1


Analyzing Product : L-Malate


,lower,upper,pfba,products_within_bounds,number_within_bounds,mode,no_mode
FRD3,0.000000,4.764980,0.000000,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",66,0.000,65
NADH18pp,0.000000,4.764819,0.000000,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",66,0.000,65
TALA,-2.360472,0.013324,0.000000,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",63,0.000,41
PFK_3,0.000000,2.373796,0.013324,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",65,0.000,23
FBA3,0.000000,2.373796,0.013324,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",65,0.000,23
PFK,0.000000,2.373796,2.360472,"[Glycolate, α-Ketoglutarate, 3-Hydroxypropanoa...",45,0.000,3
FBA,0.000000,2.373796,2.360472,"[Glycolate, α-Ketoglutarate, 3-Hydroxypropanoa...",44,0.000,2
NADH16pp,0.000000,4.764819,4.763819,"[Glycolate, Acetate, Pyruvate, L-Idonic Acid, ...",7,2.765,1
SUCDi,0.850372,5.617351,0.850475,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",24,0.912,1


Analyzing Product : L-Threonine


...

,lower,upper,pfba,products_within_bounds,number_within_bounds,mode,no_mode
FRD3,0.000000,10.002294,0.000000,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",66,0.000,65
NADH18pp,0.000000,10.001978,0.000000,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",66,0.000,65
TALA,-1.860904,1.106121,0.000000,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",65,0.000,41
PFK_3,0.000000,2.967025,1.106121,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",66,0.000,23
FBA3,0.000000,2.967025,1.106121,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",66,0.000,23
PFK,0.000000,2.967025,1.860904,"[L-Phenylalanine, Glycolate, α-Ketoglutarate, ...",57,0.000,3
FBA,0.000000,2.967025,1.860904,"[L-Phenylalanine, Glycolate, α-Ketoglutarate, ...",56,0.000,2
NADH16pp,0.000000,10.001978,10.001978,"[(R)-Propanediol, Glycolate, α-Ketoglutarate, ...",42,2.765,1
SUCDi,0.020249,10.024543,0.020454,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",49,0.088,1


Analyzing Product : γ-Aminobutyrate


,lower,upper,pfba,products_within_bounds,number_within_bounds,mode,no_mode
FRD3,0.000000,8.006294,0.000000,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",66,0.000,65
NADH18pp,0.000000,8.006180,0.000000,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",66,0.000,65
PFK_3,0.000000,3.972876,0.000000,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",66,0.000,22
FBA3,0.000000,3.972876,0.000000,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",66,0.000,22
SUCDi,0.000000,8.008407,0.000000,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",66,0.000,16
PFK,0.000000,3.972876,3.972876,"[L-Phenylalanine, Glycolate, α-Ketoglutarate, ...",64,0.000,3
FBA,0.000000,3.972876,3.972876,"[L-Phenylalanine, Glycolate, α-Ketoglutarate, ...",63,0.000,2
NADH16pp,0.000000,8.006180,8.006180,"[(R)-Propanediol, Glycolate, α-Ketoglutarate, ...",30,2.765,1
TALA,-4.097613,-0.124737,-0.124737,"[Glycolate, α-Ketoglutarate, Cytidine, Acetate...",9,-0.962,1


Analyzing Product : L-Proline


,lower,upper,pfba,products_within_bounds,number_within_bounds,mode,no_mode
FRD3,0.00000,10.440522,0.000000,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",66,0.000,65
NADH18pp,0.00000,10.440376,0.000000,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",66,0.000,65
TALA,-2.86391,1.061850,0.000000,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",65,0.000,41
PFK_3,0.00000,3.925759,1.061850,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",66,0.000,23
FBA3,0.00000,3.925759,1.061850,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",66,0.000,23
SUCDi,0.00000,10.442668,0.000000,"[L-Phenylalanine, (R)-Propanediol, Glycolate, ...",66,0.000,16
PFK,0.00000,3.925759,2.863910,"[L-Phenylalanine, Glycolate, α-Ketoglutarate, ...",63,0.000,3
FBA,0.00000,3.925759,2.863910,"[L-Phenylalanine, Glycolate, α-Ketoglutarate, ...",62,0.000,2
NADH16pp,0.00000,10.440376,10.440376,"[(R)-Propanediol, Glycolate, α-Ketoglutarate, ...",43,2.765,1


In [94]:
# Deepcopy parsed_fva and fva to perform product specific constrained fva
temp_fva = deepcopy(fva_fourth)
temp_parsed = deepcopy(parsed_fva_4)



In [96]:
deletable_reactions = {}
for product in fva_fourth.keys():
    temp = temp_dict[product][temp_dict[product]['mode']==0].index
    for reaction in temp:
        if (parsed_fva_4['flexible']['bounds'][product].loc[reaction]['lb']<=0) and (parsed_fva_4['flexible']['bounds'][product].loc[reaction]['ub']>=0):
            deletable_reactions[reaction] = deletable_reactions.get(reaction,[])+[product]
df_dict = {'products':[products for products in deletable_reactions.values()],
           'no':[len(parsed_fva_4['flexible']['pfba'].loc[reaction][parsed_fva_4['flexible']['pfba'].loc[reaction].round(decimals=3)==0]) for reaction in deletable_reactions.keys()]}
deletable_reactions = pd.DataFrame(data=df_dict,index=deletable_reactions.keys())
deletable_reactions = deletable_reactions.sort_values(by='no',ascending=False)

In [97]:
deletable_reactions

,products,no
FRD3,"[L-Tryptophan, Thymidine, L-Phenylalanine, Ade...",66
NADH18pp,"[L-Tryptophan, Thymidine, L-Phenylalanine, Ade...",66
FTHFD,[(R)-Propanediol],65
FDH4pp,[(R)-Propanediol],65
MALS,"[L-Idonic Acid, 5-Ketogluconate, (R)-Propanediol]",63
ICL,"[L-Idonic Acid, 5-Ketogluconate, (R)-Propanediol]",61
THD2pp,[Dihydroxyacetone],48
TALA,"[L-Tryptophan, Thymidine, L-Phenylalanine, L-T...",42
FBA3,"[L-Tryptophan, Thymidine, L-Phenylalanine, Ade...",23
PFK_3,"[L-Tryptophan, Thymidine, L-Phenylalanine, Ade...",23


In [100]:
# FIFTH PASS FVA -  IMPLEMENT PRODUCT SPECIFIC DELETIONS OR ZERO CONSTRAINTS
# 1. For each flexible product:
# 2. Filter reactions that have '0' as their most frequently occurring flux value
# 3. For each such reaction:
# 4. if 0 is in the flux range for that product:
# 5. Perform FVA with new constraints for that specific product
# 6. Update fva, parsed_fva, perform the 
# 7. Continue
prod_specific_deletions = {}
for i,product in enumerate(fva_fourth.keys()):
    print('Evaluating product ', i+1, ' of 70')
    for reaction in deletable_reactions.index:
        if reaction in temp_parsed['flexible']['products'].index:
            if ((temp_parsed['flexible']['bounds'][product].loc[reaction]['lb']<=0) and (temp_parsed['flexible']['bounds'][product].loc[reaction]['ub']>=0) and temp_parsed['flexible']['range'][product].loc[reaction]>0):
                print('Removing reaction: ',reaction)
                prod_specific_deletions[product]=prod_specific_deletions.get(product,[]) + [reaction]
                temp = reduced_fva(flux_dict_yield_2_prod, product, unused_reactions=temp_parsed['unused_reactions_set'].union(set(prod_specific_deletions.get(product))),
                                    used_reactions=temp_parsed['used_reactions_set'], template_model=model, slack=0.001)
                temp_fva[product]=temp[product]
                temp_fva,temp_parsed = global_fva_parser(temp_fva,flex_cutoff=0.008)



Evaluating product  1  of 70
Removing reaction:  FRD3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpq79gx96u.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  TALA
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpc6c89u06.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  2  of 70
Removing reaction:  FRD3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpddl8ttoc.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  TALA
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpz35dh1tu.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  3  of 70
Removing reaction:  FRD3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmplvbx5145.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  TALA
Read LP forma

Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  22  of 70
Removing reaction:  FRD3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp9kuuaiug.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  TALA
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpx2usng7k.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  23  of 70
Removing reaction:  FRD3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp7q5klp4h.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  TALA
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpkj8dw5_l.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  24  of 70
Removing reaction:  FRD3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpmiwotocc.lp
Reading time = 0.03 seconds
: 1805 

: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  FBA3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpoqkttv13.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  44  of 70
Removing reaction:  FRD3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpo144xgai.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  TALA
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp9ws1g350.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  45  of 70
Removing reaction:  FRD3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp7cmk5so0.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  TALA
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpb3ctfh8_.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  46 

Removing reaction:  FBA3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp6tp_l0lj.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  65  of 70
Removing reaction:  FRD3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp59isf9pf.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  FBA3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpv8ub2l2r.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  66  of 70
Removing reaction:  FRD3
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmp0_xismi9.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Removing reaction:  TALA
Read LP format model from file C:\Users\kraj5\AppData\Local\Temp\tmpthtpc1ya.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Evaluating product  67  of 70
Removing reaction:  FRD3
Read LP fo

In [101]:
fva_fifth = deepcopy(temp_fva)

In [102]:
pickling_on = open("fva_fifth_yield_2_prod.pickle","wb")
pickle.dump(fva_fifth, pickling_on)
pickling_on.close() 

In [134]:
pickle_off = open("fva_fifth_yield_2_prod.pickle","rb")
fva_fifth = pickle.load(pickle_off)
pickle_off.close()

In [135]:
reactions_set = set([reaction.id for reaction in model.reactions])
relevant_reactions_set = set(relevant_reactions)
irrelevant_reactions_set = reactions_set-relevant_reactions_set

fva_fifth,parsed_fva_5 = global_fva_parser(fva_fifth,flex_cutoff=0.015)

print("Total Reactions: ",len(reactions_set), "\n")
print("Irrelevant_reactions: ",len(irrelevant_reactions_set))
print("Relevant Reactions: ",len(relevant_reactions_set), "\n")
print("Inflexible Reactions: ",len(parsed_fva_5['inflexible_reactions_set']))
print("Unused Inflexible Reactions: ",len(parsed_fva_5['inflexible_reactions_set'].intersection(parsed_fva_5['unused_reactions_set'])))
print("Used Inflexible Reactions: ",len(parsed_fva_5['inflexible_reactions_set'].intersection(parsed_fva_5['used_reactions_set'])), "\n")
print("Flexible Reactions: ",len(parsed_fva_5['flexible_reactions_set']))
print("Unused Flexible Reactions: ",len(parsed_fva_5['flexible_reactions_set'].intersection(parsed_fva_5['unused_reactions_set'])))
print("Used Flexible Reactions: ",len(parsed_fva_5['flexible_reactions_set'].intersection(parsed_fva_5['used_reactions_set'])))


Total Reactions:  2583 

Irrelevant_reactions:  1090
Relevant Reactions:  1493 

Inflexible Reactions:  1492
Unused Inflexible Reactions:  1123
Used Inflexible Reactions:  369 

Flexible Reactions:  1
Unused Flexible Reactions:  0
Used Flexible Reactions:  1


In [137]:
min_biomass = min(parsed_fva_5['inflexible']['pfba'].loc['BIOMASS_Ec_iJO1366_core_53p95M'])
for product in fva_fifth.keys():
    cutoff = 0.001*parsed_fva_5['inflexible']['pfba'][product].loc['BIOMASS_Ec_iJO1366_core_53p95M']/min_biomass
    fva_fifth[product]['pfba'][abs(fva_fifth[product]['pfba'])<=cutoff]=0
    fva_fifth[product]['lb'][abs(fva_fifth[product]['lb'])<=cutoff]=0
    fva_fifth[product]['ub'][abs(fva_fifth[product]['ub'])<=cutoff]=0
    fva_fifth[product]['range'] = fva_fifth[product]['ub'] - fva_fifth[product]['lb']
    fva_fifth[product]['range'][abs(fva_fifth[product]['range'])<=cutoff]=0

In [138]:
reactions_set = set([reaction.id for reaction in model.reactions])
relevant_reactions_set = set(relevant_reactions)
irrelevant_reactions_set = reactions_set-relevant_reactions_set

fva_final,parsed_fva_final = global_fva_parser(fva_fifth,flex_cutoff=0.019)

print("Total Reactions: ",len(reactions_set), "\n")
print("Irrelevant_reactions: ",len(irrelevant_reactions_set))
print("Relevant Reactions: ",len(relevant_reactions_set), "\n")
print("Inflexible Reactions: ",len(parsed_fva_final['inflexible_reactions_set']))
print("Unused Inflexible Reactions: ",len(parsed_fva_final['inflexible_reactions_set'].intersection(parsed_fva_final['unused_reactions_set'])))
print("Used Inflexible Reactions: ",len(parsed_fva_final['inflexible_reactions_set'].intersection(parsed_fva_final['used_reactions_set'])), "\n")
print("Flexible Reactions: ",len(parsed_fva_final['flexible_reactions_set']))
print("Unused Flexible Reactions: ",len(parsed_fva_final['flexible_reactions_set'].intersection(parsed_fva_final['unused_reactions_set'])))
print("Used Flexible Reactions: ",len(parsed_fva_final['flexible_reactions_set'].intersection(parsed_fva_final['used_reactions_set'])))


Total Reactions:  2583 

Irrelevant_reactions:  1090
Relevant Reactions:  1493 

Inflexible Reactions:  1493
Unused Inflexible Reactions:  1147
Used Inflexible Reactions:  346 

Flexible Reactions:  0
Unused Flexible Reactions:  0
Used Flexible Reactions:  0


In [139]:
pickling_on = open("fva_final_yield_2_prod.pickle","wb")
pickle.dump(fva_final, pickling_on)
pickling_on.close() 

In [6]:
pickle_off = open("fva_final_yield_2_prod.pickle","rb")
fva_final = pickle.load(pickle_off)
pickle_off.close()

In [141]:
pickling_on = open("parsed_fva_final_yield_2_prod.pickle","wb")
pickle.dump(parsed_fva_final, pickling_on)
pickling_on.close() 

In [7]:
pickle_off = open("parsed_fva_final_yield_2_prod.pickle","rb")
parsed_fva_final = pickle.load(pickle_off)
pickle_off.close()

In [143]:
parsed_fva_final['flexible']['products']

,flex,flex_no,inflex_used,inflex_used_no,inflex_unused,inflex_unused_no


In [144]:
len(parsed_fva_final['used_reactions_set'])

346

In [145]:
parsed_fva_final['inflexible']['pfba']

,L-Tryptophan,Thymidine,L-Phenylalanine,Adenosine,L-Tyrosine,Inosine,Xanthosine,Cytidine,Uridine,Quinate,L-Isoleucine,L-Lysine,Hexanoate,L-Leucine,L-Arginine,L-Histidine,D-Gluconic Acid,L-Idonic Acid,5-Ketogluconate,Citric Acid,Agmatine,Ornithine,L-Proline,L-Valine,Thymine,Adenine,Guanine,Hypoxanthine,O-Acetyl-L-serine,Xanthine,L-Glutamate,α-Ketoglutarate,Putrescine,L-Threonine,γ-Aminobutyrate,L-Homoserine,Allantoin,Succinate,Uracil,L-Asparagine,L-Malate,L-Aspartate,L-Cysteine,Glycerol3-phosphate,3-Hydroxypropanoate,(S)-Propanediol,(R)-Propanediol,D-Alanine,Glycerol,(R)-Glycerate,Glyceraldehyde,L-Lactate,Dihydroxyacetone,L-Alanine,D-Lactate,L-Serine,Pyruvate,Ethanolamine,Ethanol,Acetaldehyde,Glycine,Acetate,Glycolate,Urea,Formate,Glutathione,Cadaverine
AMPN,0.000000,0.000000,0.000000,1.265053,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.461286,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ALATA_L,-0.182601,-0.095474,-0.173446,-0.086782,-0.127024,-0.082863,-0.105162,-0.080613,-0.081025,-0.082387,-0.175428,-0.115189,-0.174021,-0.122580,-0.085320,-0.091095,-0.062854,-0.064743,-0.112724,-0.050850,-0.133455,-0.097201,-0.126866,-0.105091,-0.103187,-0.084364,-0.074073,-0.080446,-0.068997,-0.066287,-0.067528,-0.061611,-0.184271,-0.083386,-0.098693,-0.079825,-0.060507,-0.063381,-0.063945,-0.054128,-0.049188,-0.048958,-0.098816,-0.063967,-0.104900,-0.120608,-0.116961,-3.725998,-0.100256,-0.070921,-0.078340,-0.080204,-0.078890,-4.017442,-0.131041,-0.053144,-0.064592,-0.180537,-0.160324,-0.118270,-0.055853,-0.067547,-0.050320,-0.059848,-0.017775,-0.091681,-0.184671
SDPDS,0.116513,0.060920,0.110672,0.055374,0.081051,0.052873,0.067101,0.051437,0.051700,0.052569,0.111937,2.252339,0.111038,0.078215,0.054441,0.058125,0.040105,0.040673,0.071926,0.032446,0.085155,0.062022,0.080950,0.067056,0.065841,0.053830,0.047264,0.051331,0.044025,0.042296,0.043088,0.039313,0.117579,0.053207,0.062974,0.050934,0.038608,0.040442,0.040802,0.034537,0.031386,0.031239,0.063052,0.040816,0.066934,0.076957,0.074630,0.062559,0.063971,0.045253,0.049987,0.051176,0.050337,0.042628,0.083614,0.033910,0.041215,0.115197,0.102299,0.075465,0.035639,0.043100,0.032108,0.038188,0.011342,0.058500,2.873150
GPDDA2pp,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.960582,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
LCARS,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.008845,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
UGMDDS,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [8]:
len(fva_final.keys())

67